In [1]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [2]:
%fs ls /FileStore/tables/raw

path,name,size
dbfs:/FileStore/tables/raw/despesas_2009-d0d22.csv,despesas_2009-d0d22.csv,73741287
dbfs:/FileStore/tables/raw/despesas_2010-91775.csv,despesas_2010-91775.csv,96572819
dbfs:/FileStore/tables/raw/despesas_2011-2289d.csv,despesas_2011-2289d.csv,107508369
dbfs:/FileStore/tables/raw/despesas_2012-8ecc3.csv,despesas_2012-8ecc3.csv,99933266
dbfs:/FileStore/tables/raw/despesas_2013-e29e4.csv,despesas_2013-e29e4.csv,108572926
dbfs:/FileStore/tables/raw/despesas_2014-fae0d.csv,despesas_2014-fae0d.csv,104301267
dbfs:/FileStore/tables/raw/despesas_2015-55418.csv,despesas_2015-55418.csv,128248244
dbfs:/FileStore/tables/raw/despesas_2016-414fe.csv,despesas_2016-414fe.csv,122285142
dbfs:/FileStore/tables/raw/despesas_2017-89277.csv,despesas_2017-89277.csv,118080465
dbfs:/FileStore/tables/raw/despesas_2018-2f77b.csv,despesas_2018-2f77b.csv,96715561


In [3]:
# Temas de proposicoes
propositions_theme_schema = StructType([
    StructField('uriProposicao', StringType()),
    StructField('siglaTipo', StringType()),
    StructField('numero', IntegerType()),
    StructField('ano', IntegerType()),
    StructField('codTema', IntegerType()),
    StructField('tema', StringType()),
    StructField('relevancia', IntegerType())
])

df_propositions_theme = spark.read.csv('/FileStore/tables/raw/proposicoesTemas*.csv', header=True, sep=';', schema=propositions_theme_schema)

df_propositions_theme = df_propositions_theme.withColumn('ID_PROPOSICAO', F.split(df_propositions_theme.uriProposicao, '/').getItem(6))

df_propositions_theme = df_propositions_theme.select(
  F.col('ID_PROPOSICAO'),
  F.col('siglaTipo').alias('DC_SIGLA_TIPO'),
  F.col('numero').alias('NU_TEMA_PROPOSICAO'),
  F.col('ano').alias('NU_ANO'),
  F.col('codTema').alias('CD_TEMA'),
  F.col('tema').alias('DC_TEMA'),
  F.col('relevancia').alias('NU_RELEVANCIA')
)

df_propositions_theme = df_propositions_theme.cache()

In [4]:
df_propositions_theme.show(truncate=False)

+-------------+-------------+------------------+------+-------+-------------------------------------------+-------------+
ID_PROPOSICAO|DC_SIGLA_TIPO|NU_TEMA_PROPOSICAO|NU_ANO|CD_TEMA|DC_TEMA |NU_RELEVANCIA|
+-------------+-------------+------------------+------+-------+-------------------------------------------+-------------+
2207000 |PL |3379 |2019 |43 |Direito Penal e Processual Penal |0 |
2210208 |EMS |3820 |2019 |64 |Agricultura, Pecuária, Pesca e Extrativismo|0 |
2210208 |EMS |3820 |2019 |58 |Trabalho e Emprego |0 |
2219227 |MSC |410 |2019 |34 |Administração Pública |0 |
2220305 |MSC |418 |2019 |34 |Administração Pública |0 |
2234000 |EMS |4805 |2019 |62 |Ciência, Tecnologia e Inovação |0 |
2234000 |EMS |4805 |2019 |37 |Comunicações |0 |
2234000 |EMS |4805 |2019 |70 |Finanças Públicas e Orçamento |0 |
2234000 |EMS |4805 |2019 |66 |Indústria, Comércio e Serviços |0 |
2236092 |EMS |6606 |2019 |42 |Direito Civil e Processual Civil |0 |
2236092 |EMS |6606 |2019 |66 |Indústria, Comércio e Serviços |0 |
2236041 |EMS |2114 |2019 |57 |Defesa e Segurança |0 |
2236083 |MPV |912 |2019 |44 |Direitos Humanos e Minorias |0 |
2236083 |MPV |912 |2019 |70 |Finanças Públicas e Orçamento |0 |
2236083 |MPV |912 |2019 |55 |Relações Internacionais e Comércio Exterior|0 |
308880 |PL |2441 |2019 |40 |Economia |0 |
317970 |PL |2037 |2019 |34 |Administração Pública |0 |
427339 |PL |2439 |2019 |66 |Indústria, Comércio e Serviços |0 |
501638 |PL |402 |2019 |44 |Direitos Humanos e Minorias |0 |
512431 |PL |2442 |2019 |37 |Comunicações |0 |
+-------------+-------------+------------------+------+-------+-------------------------------------------+-------------+
only showing top 20 rows

In [5]:
# Proposicoes
propositions_schema = StructType([
    StructField('id', IntegerType()),
    StructField('uri', StringType()),
    StructField('siglaTipo', StringType()),
    StructField('numero', IntegerType()),
    StructField('ano', IntegerType()),
    StructField('codTipo', IntegerType()),
    StructField('descricaoTipo', StringType()),
    StructField('ementa', StringType()),
    StructField('ementaDetalhada', StringType()),
    StructField('keywords', StringType()),
    StructField('dataApresentacao', StringType()),
    StructField('uriOrgaoNumerador', StringType()),
    StructField('uriPropAnterior', StringType()),
    StructField('uriPropPrincipal', StringType()),
    StructField('uriPropPosterior', StringType()),
    StructField('urlInteiroTeor', StringType()),
    StructField('urnFinal', StringType()),
    StructField('ultimoStatus_dataHora', StringType()),
    StructField('ultimoStatus_sequencia', IntegerType()),
    StructField('ultimoStatus_uriRelator', StringType()),
    StructField('ultimoStatus_idOrgao', IntegerType()),
    StructField('ultimoStatus_siglaOrgao', StringType()),
    StructField('ultimoStatus_uriOrgao', StringType()),
    StructField('ultimoStatus_regime', StringType()),
    StructField('ultimoStatus_descricaoTramitacao', StringType()),
    StructField('ultimoStatus_idTipoTramitacao', IntegerType()),
    StructField('ultimoStatus_descricaoSituacao', StringType()),
    StructField('ultimoStatus_idSituacao', IntegerType()),
    StructField('ultimoStatus_despacho', StringType()),
    StructField('ultimoStatus_url', StringType())
])

df_propositions = spark.read.csv('/FileStore/tables/raw/proposicoes_*.csv', header=True, sep=';', schema=propositions_schema)

df_propositions = df_propositions.cache()

df_propositions = df_propositions.select(
    F.col('id').alias('ID_PROPOSICAO'),
    F.col('siglaTipo').alias('SG_TIPO'),
    F.col('numero').alias('NU_PROPOSICAO'),
    F.col('ano').alias('NU_ANO'),
    F.col('codTipo').alias('CD_TIPO'),
    F.col('descricaoTipo').alias('DC_TIPO'),
    F.col('ementa').alias('DC_EMENTA'),
    F.col('ementaDetalhada').alias('DC_EMENTA_DETALHADA'),
    F.col('keywords').alias('DC_KEYWORDS'),
    F.col('dataApresentacao').alias('DT_APRESENTACAO')
)

In [6]:
df_propositions.show()

+-------------+-------+-------------+------+-------+------------------+--------------------+--------------------+--------------------+-------------------+
ID_PROPOSICAO|SG_TIPO|NU_PROPOSICAO|NU_ANO|CD_TIPO| DC_TIPO| DC_EMENTA| DC_EMENTA_DETALHADA| DC_KEYWORDS| DT_APRESENTACAO|
+-------------+-------+-------------+------+-------+------------------+--------------------+--------------------+--------------------+-------------------+
 865934| MPV| 667| 2015| 291| Medida Provisória|Abre crédito extr...| null|Crédito extraordi...|2015-01-05T13:07:00|
 865943| PRL| 3| 0| 190|Parecer do Relator|Parecer da Relato...| null| null|2015-01-08T11:35:00|
 865944| SBT| 1| 0| 255| Substitutivo|Acresce inciso ao...| null| null|2015-01-08T11:37:00|
 865949| PRL| 1| 0| 190|Parecer do Relator|Parecer do Relato...| null| null|2015-01-12T10:13:00|
 865950| PRL| 1| 0| 190|Parecer do Relator|Parecer do Relato...| null| null|2015-01-12T10:14:00|
 865951| SBT| 1| 0| 255| Substitutivo|Estabelece a Polí...| null| null|2015-01-12T10:20:00|
 865953| PRL| 1| 0| 190|Parecer do Relator|Parecer do Relato...| null| null|2015-01-12T11:22:00|
 865959| PL| 8328| 2015| 139| Projeto de Lei|Altera a Lei n° 1...|"Substitui a expr...|Alteração, Lei da...|2015-01-09T17:43:00|
 865960| PL| 8329| 2015| 139| Projeto de Lei|Altera a Lei n° 1...| null|Alteração, Estatu...|2015-01-09T17:43:00|
 865961| PL| 8330| 2015| 139| Projeto de Lei|"Altera a Lei n° ...| null|Alteração, Lei Ma...|2015-01-09T17:43:00|
 865962| PL| 8331| 2015| 139| Projeto de Lei|"Altera a Lei n° ...| null|Alteração, Estatu...|2015-01-09T17:43:00|
 865965| PRL| 3| 0| 190|Parecer do Relator|Parecer do relato...| null| null|2015-01-13T14:27:00|
 865970| PRL| 1| 0| 190|Parecer do Relator|Parecer da Relato...| null| null|2015-01-14T17:03:00|
 865971| SBT| 1| 0| 255| Substitutivo|Dispõe sobre o in...| null| null|2015-01-14T17:09:00|
 865973| PRL| 2| 0| 190|Parecer do Relator|Parecer do Relato...| null| null|2015-01-15T15:56:00|
 865977| PRL| 1| 0| 190|Parecer do Relator|Parecer do Relato...| null| null|2015-01-20T10:05:00|
 865980| PRL| 3| 0| 190|Parecer do Relator|Parecer do Relato...| null| null|2015-01-21T11:38:00|
 865981| PRL| 7| 0| 190|Parecer do Relator|Parecer do Relato...| null| null|2015-01-21T11:38:00|
 865982| PRL| 2| 0| 190|Parecer do Relator|Parecer do Relato...| null| null|2015-01-21T11:39:00|
 865983| PRL| 5| 0| 190|Parecer do Relator|Parecer do Relato...| null| null|2015-01-21T11:40:00|
+-------------+-------+-------------+------+-------+------------------+--------------------+--------------------+--------------------+-------------------+
only showing top 20 rows

In [7]:
# Despesas
expenses_schema = StructType([
    StructField('txtNomeParlamentar', StringType()),
    StructField('cpf', StringType()),
    StructField('ideCadastro', StringType()),
    StructField('nuCarteiraParlamentar', StringType()),
    StructField('nuLegislatura', IntegerType()),
    StructField('sgUF', StringType()),
    StructField('sgPartido', StringType()),
    StructField('codLegislatura', StringType()),
    StructField('numSubCota', IntegerType()),
    StructField('txtDescricao', StringType()),
    StructField('numEspecificacaoSubCota', IntegerType()),
    StructField('txtDescricaoEspecificacao', StringType()),
    StructField('txtFornecedor', StringType()),
    StructField('txtCNPJCPF', StringType()),
    StructField('txtNumero', StringType()),
    StructField('indTipoDocumento', StringType()),
    StructField('datEmissao', StringType()),
    StructField('vlrDocumento', DoubleType()),
    StructField('vlrGlosa', DoubleType()),
    StructField('vlrLiquido', DoubleType()),
    StructField('numMes', IntegerType()),
    StructField('numAno', IntegerType()),
    StructField('numParcela', IntegerType()),
    StructField('txtPassageiro', StringType()), 
    StructField('txtTrecho', StringType()), 
    StructField('numLote', IntegerType()), 
    StructField('numRessarcimento', IntegerType()), 
    StructField('vlrRestituicao', DoubleType()), 
    StructField('nuDeputadoId', IntegerType()), 
    StructField('ideDocumento', IntegerType()), 
    StructField('urlDocumento', StringType())
])

df_expenses = spark.read.csv('/FileStore/tables/raw/despesas*.csv', header=True, sep=';', schema=expenses_schema)

df_expenses = df_expenses.where(F.col('ideCadastro').isNotNull())

df_expenses = df_expenses.cache()

df_politicians = df_expenses.select(
  F.col('ideCadastro').alias('ID_DEPUTADO'),
  F.col('txtNomeParlamentar').alias('NM_PARLAMENTAR'),
  F.col('cpf').alias('DC_CPF_PARLAMENTAR'),
  F.col('nuCarteiraParlamentar').alias('NU_CARTEIRA_PARLAMENTAR'),
  F.col('nuLegislatura').alias('NU_LEGISLATURA'),
  F.col('sgUF').alias('SG_UF'),
  F.col('sgPartido').alias('SG_PARTIDO'),
  F.col('codLegislatura').alias('CD_LEGISLATURA')
).drop_duplicates()

df_expenses = df_expenses.select(
  F.col('ideCadastro').alias('ID_DEPUTADO'),
  F.col('numSubCota').alias('NU_SUBCOTA'),
  F.col('txtDescricao').alias('DC_DESCRICAO'),
  F.col('numEspecificacaoSubCota').alias('NU_ESPECIFICACAO_SUBCOTA'),
  F.col('txtDescricaoEspecificacao').alias('DC_ESPECIFICACAO'),
  F.col('txtFornecedor').alias('NM_FORNECEDOR'),
  F.col('txtCNPJCPF').alias('DC_CPFCNPJ_FORNECEDOR'),
  F.col('txtNumero').alias('DC_NUMERO'),
  F.col('indTipoDocumento').alias('DC_TIPO_DOCUMENTO'),
  F.col('datEmissao').alias('DT_EMISSAO'),
  F.col('vlrDocumento').alias('VL_DOCUMENTO'),
  F.col('vlrGlosa').alias('VL_GLOSA'),
  F.col('vlrLiquido').alias('VL_LIQUIDO'),
  F.col('numMes').alias('NU_MES'),
  F.col('numAno').alias('NU_ANO'),
  F.col('numParcela').alias('NU_PARCELA'),
  F.col('txtPassageiro').alias('DC_PASSAGEIRO'), 
  F.col('txtTrecho').alias('DC_TRECHO'), 
  F.col('numLote').alias('NU_LOTE'), 
  F.col('numRessarcimento').alias('NU_RESSARCIMENTO'), 
  F.col('vlrRestituicao').alias('VL_RESTITUICAO'), 
  F.col('ideDocumento').alias('ID_DOCUMENTO')
)

In [8]:
df_politicians.show()

+-----------+-------------------+------------------+-----------------------+--------------+-----+----------+--------------+
ID_DEPUTADO| NM_PARLAMENTAR|DC_CPF_PARLAMENTAR|NU_CARTEIRA_PARLAMENTAR|NU_LEGISLATURA|SG_UF|SG_PARTIDO|CD_LEGISLATURA|
+-----------+-------------------+------------------+-----------------------+--------------+-----+----------+--------------+
 73434|ARNALDO FARIA DE SÁ| 21911452800| 337| 2007| SP| PTB| 53|
 73478| BETO ALBUQUERQUE| 33769460006| 490| 2011| RS| PSB| 54|
 64960| MARIA HELENA| 11251778291| 6| 2007| RR| PSB| 53|
 73416| NELSON BORNIER| 10041800710| 549| 2007| RJ| PMDB| 53|
 3151| JAIRO ATAÍDE| 03428311604| 533| 2007| MG| DEM| 53|
 67312| JÚLIA MARINHO| 10799397253| 34| 2015| PA| PSC| 55|
 69871| BACELAR| 10626409500| 184| 2015| BA| PODE| 55|
 73460| GUSTAVO FRUET| 64446379968| 452| 2007| PR| PSDB| 53|
 3155| BISPO GÊ TENUTA| null| 526| 2011| SP| DEM| 54|
 73474| LUCI CHOINACKI| 70740178920| 477| 2011| SC| PT| 54|
 73434|ARNALDO FARIA DE SÁ| 21911452800| 337| 2015| SP| PP| 55|
 73437| BETO MANSUR| 73255319820| 340| 2007| SP| PP**| 53|
 4927| JORGINHO MALULY| null| 517| 2011| SP| DEM| 54|
 73437| BETO MANSUR| 73255319820| 340| 2011| SP| PRB| 54|
 73459| DR. ROSINHA| 29938520944| 449| 2011| PR| PT| 54|
 73424| SIMÃO SESSIM| 03441067720| 327| 2007| RJ| PP**| 53|
 73458| ALEX CANZIANI| 36601101915| 445| 2015| PR| PTB| 55|
 73433| ARLINDO CHINAGLIA| 06821146187| 336| 2015| SP| PT| 55|
 67138| IRACEMA PORTELLA| 37311638372| 113| 2015| PI| PP| 55|
 67138| IRACEMA PORTELLA| 37311638372| 113| 2019| PI| PP| 56|
+-----------+-------------------+------------------+-----------------------+--------------+-----+----------+--------------+
only showing top 20 rows

In [9]:
df_expenses.show()

+-----------+----------+--------------------+------------------------+--------------------+--------------------+---------------------+---------------+-----------------+-------------------+------------+--------+----------+------+------+----------+-------------+---------+-------+----------------+--------------+------------+
ID_DEPUTADO|NU_SUBCOTA| DC_DESCRICAO|NU_ESPECIFICACAO_SUBCOTA| DC_ESPECIFICACAO| NM_FORNECEDOR|DC_CPFCNPJ_FORNECEDOR| DC_NUMERO|DC_TIPO_DOCUMENTO| DT_EMISSAO|VL_DOCUMENTO|VL_GLOSA|VL_LIQUIDO|NU_MES|NU_ANO|NU_PARCELA|DC_PASSAGEIRO|DC_TRECHO|NU_LOTE|NU_RESSARCIMENTO|VL_RESTITUICAO|ID_DOCUMENTO|
+-----------+----------+--------------------+------------------------+--------------------+--------------------+---------------------+---------------+-----------------+-------------------+------------+--------+----------+------+------+----------+-------------+---------+-------+----------------+--------------+------------+
 3151| 1|MANUTENÇÃO DE ESC...| 0| null|CEMIG DISTRIBUIÇA...| 069.811.800/0011-6| 001562795| 0|2015-01-13T00:00:00| 13.19| 0.25| 12.94| 1| 2015| 0| null| null|1169573| null| null| 5611810|
 3151| 1|MANUTENÇÃO DE ESC...| 0| null|CEMIG DISTRIBUIÇA...| 069.811.800/0011-6| 001562796| 0|2015-01-13T00:00:00| 90.43| 1.3| 89.13| 1| 2015| 0| null| null|1163724| null| null| 5591461|
 3151| 1|MANUTENÇÃO DE ESC...| 0| null| COPASA| 172.811.060/0010-3| 00115027414617| 0|2015-01-14T00:00:00| 57.8| 0.99| 56.81| 1| 2015| 0| null| null|1163724| null| null| 5591446|
 3151| 3|COMBUSTÍVEIS E LU...| 1|Veículos Automotores|CASCOL COMBUSTÍVE...| 003.065.970/0630-8| 014297| 0|2015-01-30T00:00:00| 162.84| 0.0| 162.84| 1| 2015| 0| null| null|1169573| null| null| 5611848|
 3151| 3|COMBUSTÍVEIS E LU...| 1|Veículos Automotores|J P DERIVADOS DE ...| 029.984.140/0018-7| 041748| 0|2015-01-29T00:00:00| 100.01| 0.0| 100.01| 1| 2015| 0| null| null|1169573| null| null| 5611851|
 3151| 3|COMBUSTÍVEIS E LU...| 1|Veículos Automotores| POSTO REM LTDA| 417.052.110/0010-7| 471379| 0|2015-01-28T00:00:00| 210.51| 0.0| 210.51| 1| 2015| 0| null| null|1169574| null| null| 5611844|
 3151| 4|CONSULTORIAS, PES...| 0| null|MENEZES - CONSULT...| 072.683.080/0016-1|201500000000192| 0|2015-02-05T00:00:00| 9500.0| 0.0| 9500.0| 1| 2015| 0| null| null|1169574| null| null| 5611920|
 3151| 5|DIVULGAÇÃO DA ATI...| 0| null| ZOOM PRODUTORA LTDA| 028.539.450/0018-1|201500000000037| 0|2015-01-30T00:00:00| 9000.0| 0.0| 9000.0| 1| 2015| 0| null| null|1169574| null| null| 5611918|
 3151| 8|SERVIÇO DE SEGURA...| 0| null| PROTEL| 429.396.450/0013-4|201500000024718| 0|2015-01-07T00:00:00| 243.0| 0.0| 243.0| 1| 2015| 0| null| null|1163725| null| null| 5591429|
 3151| 10| TELEFONIA| 0| null| CELULAR FUNCIONAL| 000.000.000/0000-1| 5882419| 0|2015-01-13T00:00:00| 39.24| 0.0| 39.24| 1| 2015| 0| null| null| 0| 0| null| 0|
 3151| 10| TELEFONIA| 0| null| CELULAR FUNCIONAL| 000.000.000/0000-1| 5884507| 0| null| 137.23| 0.0| 137.23| 1| 2015| 0| null| null| 0| 0| null| 0|
 3151| 10| TELEFONIA| 0| null| RAMAL| 000.000.000/0000-6| 5582889| 0| null| 91.0| 0.0| 91.0| 1| 2015| 0| null| null| 0| 0| null| 0|
 3151| 10| TELEFONIA| 0| null| RAMAL| 000.000.000/0000-6| 5590524| 0| null| 84.81| 0.0| 84.81| 1| 2015| 0| null| null| 0| 0| null| 0|
 3151| 10| TELEFONIA| 0| null| RAMAL| 000.000.000/0000-6| 5590543| 0| null| 95.45| 0.0| 95.45| 1| 2015| 0| null| null| 0| 0| null| 0|
 3151| 10| TELEFONIA| 0| null| RAMAL| 000.000.000/0000-6| 5596457| 0| null| 163.6| 0.0| 110.56| 1| 2015| 0| null| null| 0| 0| null| 0|
 3151| 11| SERVIÇOS POSTAIS| 0| null|CORREIOS - ENCOME...| 000.000.000/0000-7| RSP : 2106731| 1|2015-01-26T11:50:02| 44.87| 0.0| 44.87| 1| 2015| 0| null| null| 0| 0| null| 0|
 3151| 11| SERVIÇOS POSTAIS| 0| null| CORREIOS - SEDEX 10| 000.000.000/0000-7| RSP : 2105557| 1|2015-01-13T15:06:21| 53.6| 0.0| 53.6| 1| 2015| 0| null| null| 0| 0| null| 0|
 3151| 11| SERVIÇOS POSTAIS| 0| null| CORREIOS - SEDEX 10| 000.000.000/0000-7| RSP : 2107314| 1|2015-01-29T12:34:04| 53.6| 0.0| 53.6| 1| 2015| 0| nu

In [10]:
# Autores das Proposicoes
propositions_authors_schema = StructType([
    StructField('idProposicao', IntegerType()),
    StructField('uriProposicao', StringType()),
    StructField('idDeputadoAutor', IntegerType()),
    StructField('uriAutor', StringType()),
    StructField('codTipoAutor', StringType()),
    StructField('tipoAutor', StringType()),
    StructField('nomeAutor', StringType()),
    StructField('siglaPartidoAutor', StringType()),
    StructField('uriPartidoAutor', StringType()),
    StructField('siglaUFAutor', StringType()),
    StructField('ordemAssinatura', IntegerType()),
    StructField('proponente', IntegerType())
])

df_propositions_authors = spark.read.csv('/FileStore/tables/raw/proposicoesAutores*.csv', header=True, sep=';', schema=propositions_authors_schema)

df_propositions_authors = df_propositions_authors.cache()

df_propositions_authors = df_propositions_authors.select(
    F.col('idProposicao').alias('ID_PROPOSICAO'),
    F.col('idDeputadoAutor').alias('ID_DEPUTADO'),
    F.col('codTipoAutor').alias('CD_TIPO_AUTOR'),
    F.col('tipoAutor').alias('TP_AUTOR'),
    F.col('nomeAutor').alias('NM_AUTOR'),
    F.col('siglaPartidoAutor').alias('SG_PARTIDO_AUTOR'),
    F.col('siglaUFAutor').alias('SG_UF_AUTOR'),
    F.col('ordemAssinatura').alias('NU_ORDEM_ASSINATURA'),
    F.col('proponente').alias('NU_PROPONENTE')
)

df_propositions_authors = df_propositions_authors.where(F.col('ID_DEPUTADO').isNotNull())

In [11]:
df_propositions_authors.show()

+-------------+-----------+-------------+--------+------------------+----------------+-----------+-------------------+-------------+
ID_PROPOSICAO|ID_DEPUTADO|CD_TIPO_AUTOR|TP_AUTOR| NM_AUTOR|SG_PARTIDO_AUTOR|SG_UF_AUTOR|NU_ORDEM_ASSINATURA|NU_PROPONENTE|
+-------------+-----------+-------------+--------+------------------+----------------+-----------+-------------------+-------------+
 2237302| 141450| 10000|Deputado| Hugo Leal| PSD| RJ| 1| 1|
 2237301| 141450| 10000|Deputado| Hugo Leal| PSD| RJ| 1| 1|
 2237300| 141450| 10000|Deputado| Hugo Leal| PSD| RJ| 1| 1|
 2237299| 204521| 10000|Deputado| Abou Anni| PSL| SP| 1| 1|
 2237298| 204375| 10000|Deputado| Leda Sadala| AVANTE| AP| 1| 1|
 2237297| 204521| 10000|Deputado| Abou Anni| PSL| SP| 1| 1|
 2237296| 204521| 10000|Deputado| Abou Anni| PSL| SP| 1| 1|
 2237295| 204375| 10000|Deputado| Leda Sadala| AVANTE| AP| 1| 1|
 2237294| 204521| 10000|Deputado| Abou Anni| PSL| SP| 1| 1|
 2237293| 141411| 10000|Deputado|Dagoberto Nogueira| PDT| MS| 1| 1|
 2237292| 204375| 10000|Deputado| Leda Sadala| AVANTE| AP| 1| 1|
 2237291| 141411| 10000|Deputado|Dagoberto Nogueira| PDT| MS| 1| 1|
 2237290| 141411| 10000|Deputado|Dagoberto Nogueira| PDT| MS| 1| 1|
 2237289| 204521| 10000|Deputado| Abou Anni| PSL| SP| 1| 1|
 2237288| 141411| 10000|Deputado|Dagoberto Nogueira| PDT| MS| 1| 1|
 2237287| 178964| 10000|Deputado| Heitor Schuch| PSB| RS| 1| 1|
 2237286| 160532| 10000|Deputado| Eli Corrêa Filho| null| null| 1| 1|
 2237285| 178964| 10000|Deputado| Heitor Schuch| PSB| RS| 1| 1|
 2237284| 160532| 10000|Deputado| Eli Corrêa Filho| null| null| 1| 1|
 2237283| 74749| 10000|Deputado| Mauro Lopes| MDB| MG| 1| 1|
+-------------+-----------+-------------+--------+------------------+----------------+-----------+-------------------+-------------+
only showing top 20 rows

In [ ]:
# Export Data to curated zone

df_propositions_theme.write.parquet('/FileStore/tables/curated/propositions_themes')
df_propositions_authors.write.parquet('/FileStore/tables/curated/propositions_authors')
df_propositions.write.parquet('/FileStore/tables/curated/propositions')
df_politicians.write.parquet('/FileStore/tables/curated/politicians')
df_expenses.write.parquet('/FileStore/tables/curated/expenses')